# pandas 进阶修炼 ｜早起Python
<br>

**pandas进阶修炼系列由公众号【早起Python & 可视化图鉴】 原创**

**转载及其他形式合作请与我们联系（微信号`sshs321`)，未经授权严禁搬运及二次创作，侵权必究！**


本习题基于 `pandas` 版本 `1.1.3`，所有内容应当在 `Jupyter Notebook` 中执行以获得最佳效果。

不同版本之间写法可能会有少许不同，如若碰到此情况，你应该学会如何自行检索解决。

## 项目简介



<br>


**<font color = '#5172F0'><font size=3.5>必读👇👇👇</font>**
    
**本习题微信公众号「<font color ='#F77802'>可以叫我才哥</font>」友情提供，授权公众号「早起Python」二次整理发布**


    
### 关于同期群分析

同期群(Cohort)的字面意思(有共同特点或举止类同的)一群人，比如不同性别，不同年龄。

在《精益数据分析》中的第2章 创业的记分牌 中介绍了三种分析方法（市场细分、同期群分析以及A/B测试），其中关于同期群分析的讨论可以帮助我们快速了解它的应用场景。

- 同期群分析：比较的是相似群体随时间的变化。



产品会随着你的开发和测试而不断迭代，这就导致在产品发布第一周就加入的用户和后来才加入的用户有着不同的体验。

比如，每个用户都会经历一个生命周期：从免费试用，到付费使用，最后停止使用。同时，在这期间里，你还在不停地对商业模式进行调整。

于是，在产品上线第一个月就“吃螃蟹”的用户势必与四个月后才加入的用户有着不同的上手体验。

这对用户流失率会有什么影响？我们用同期群分析来寻找答案。



每一组用户构成一个同期群，参与整个试验过程。通过比较不同的同期群，你可以获知：从总体上看，关键指标的表现是否越来越好了。

结合到用户分析层面，比如不同月份获取的用户，不同渠道新增用户，具备不同特征的用户（比如微信里每天至少和10个以上朋友微信的用户）。

同期群分析(Cohort Analysis)，将这些具有不同特征的人群进行对比分析，以发现他们在时间维度下的行为差异。

因此，同期群分析主要用于以下2点：

- **对比** 不同 同期群群体同一体验周期的数据指标，验证产品迭代优化的效果
- **对比** 同一 同期群群体不同体验周期（生命周期）的数据指标，发现长线体验的问题

我们在进行同期群分析的时候，大致可以划分为2个流程：确定同期群分组逻辑和确定同期群分析的关键数据指标。

关于分组逻辑，需要遵循以下2个准则：

- **具有相似行为特征的群体**
- **具有相同时间周期的群体**

例如：

- **按获客月份（按周甚至按天分组）**
- **按获客渠道**
- **按照用户完成的特定行为，比如用户访问网站的次数或者购买次数来分类。**


关于关键数据指标，需要是基于时间维度下的比如留存、营收、自传播系数等等。


### 项目说明

下面是某电商的运营数据，我们将以该数据演示用 Python 进行同期群分析。

### 数据说明

数据是某电商用户付费日志，日志字段包含日期、付费金额和用户id，已脱敏处理。
    
**<font color = '#5172F0'><font size=3.5>更多资源👇👇👇</font>**   
    
如果需要获得与本案例更多的资源，可以微信搜索公众号「<font color ='#F77802'>可以叫我才哥</font>」关注，或者点击下方文章链接查看！

- [使用Python进行同期群分析](https://mp.weixin.qq.com/s/whx4lYpZVdeZTIQSvZkm2A)

### 特别说明

<br>

该 `Notebook` 版本为**习题+答案版**

因本文项目较为复杂，且部分习题很难正确表述，因此答案将直接附在习题下方。

你应该思考是否有更优解。

## 数据导入与处理

### 导入数据

In [94]:
import pandas as pd

df = pd.read_csv('日志.csv', encoding="gb18030")
df.head()

,日期,付费金额,uid
0,2021/4/30 9:50,300,9734668
1,2021/4/30 9:42,150,9947799
2,2021/4/30 9:41,680,9058431
3,2021/4/30 9:32,30,9947799
4,2021/4/30 9:25,150,2412798


### 数据重采样

因为我们是按照月份进行分组，所以需要先将日期重采样为月份：

In [95]:
df['购买月份'] = pd.to_datetime(df.日期).dt.to_period("M")
df.head()

,日期,付费金额,uid,购买月份
0,2021/4/30 9:50,300,9734668,2021-04
1,2021/4/30 9:42,150,9947799,2021-04
2,2021/4/30 9:41,680,9058431,2021-04
3,2021/4/30 9:32,30,9947799,2021-04
4,2021/4/30 9:25,150,2412798,2021-04


### 数据计算

计算每个用户在每个月的付费总额

In [96]:
order = df.groupby(["uid", "购买月份"], as_index=False).agg(
    月付费总额=("付费金额", "sum"),
    月付费次数=("uid", "count"),
)
order.head()

,uid,购买月份,月付费总额,月付费次数
0,1987205,2020-12,300,1
1,1988635,2020-12,30,1
2,2012214,2020-12,8250,12
3,2154255,2020-12,300,1
4,2155132,2020-12,300,1


### 数据计算

计算每个用户的首单购买月份作为同期群分组，并将其对应到原始数据上：

In [97]:
order["同期群分组"] = order.groupby("uid")['购买月份'].transform("min")
order.head()

,uid,购买月份,月付费总额,月付费次数,同期群分组
0,1987205,2020-12,300,1,2020-12
1,1988635,2020-12,30,1,2020-12
2,2012214,2020-12,8250,12,2020-12
3,2154255,2020-12,300,1,2020-12
4,2155132,2020-12,300,1,2020-12


### 数据计算

计算每条购买记录的时间与首单购买时间的月份差，并重置月份差标签

In [98]:
order["标签"] = (order.购买月份-order.同期群分组).apply(lambda x:"本月新增" if x.n==0 else f"+{x.n}月")
order.head()

,uid,购买月份,月付费总额,月付费次数,同期群分组,标签
0,1987205,2020-12,300,1,2020-12,本月新增
1,1988635,2020-12,30,1,2020-12,本月新增
2,2012214,2020-12,8250,12,2020-12,本月新增
3,2154255,2020-12,300,1,2020-12,本月新增
4,2155132,2020-12,300,1,2020-12,本月新增


## 同期群分析 - 从留存率角度


前面我们说了至少有3个数据指标可以进行分析：

- 留存率
- 人均付款金额
- 人均购买次数

### 数据透视

通过数据透视表可以一次性计算所需的数据

In [99]:
cohort_number = order.pivot_table(index="同期群分组", columns="标签",
                             values="uid", aggfunc="count",
                             fill_value=0).rename_axis(columns="留存率")
cohort_number

留存率,+1月,+2月,+3月,+4月,本月新增
同期群分组,,,,,
2020-12,1649,811,296,189,22870
2021-01,678,122,75,0,6330
2021-02,287,86,0,0,5210
2021-03,208,0,0,0,3407
2021-04,0,0,0,0,2445


### 移动列

将 本月新增 列移动到第一列

具体过程是先通过pop删除该列，然后插入到0位置，并命名为指定的列名。

In [100]:
cohort_number.insert(0, "本月新增", cohort_number.pop("本月新增"))
cohort_number

留存率,本月新增,+1月,+2月,+3月,+4月
同期群分组,,,,,
2020-12,22870,1649,811,296,189
2021-01,6330,678,122,75,0
2021-02,5210,287,86,0,0
2021-03,3407,208,0,0,0
2021-04,2445,0,0,0,0


### 数据计算

在本次的分析中，留存率的具体计算方式为：+N月留存率=+N月付款用户数/首月付款用户数

In [101]:
cohort_number.iloc[:, 1:] = cohort_number.iloc[:, 1:].divide(cohort_number.本月新增, axis=0)
cohort_number

留存率,本月新增,+1月,+2月,+3月,+4月
同期群分组,,,,,
2020-12,22870,0.072103,0.035461,0.012943,0.008264
2021-01,6330,0.107109,0.019273,0.011848,0.000000
2021-02,5210,0.055086,0.016507,0.000000,0.000000
2021-03,3407,0.061051,0.000000,0.000000,0.000000
2021-04,2445,0.000000,0.000000,0.000000,0.000000


### 个性化显示

以百分比形式显示，并设置颜色：

In [102]:
out1 = (cohort_number.style
    .format("{:.2%}", subset=cohort_number.columns[1:])
    .bar(subset="本月新增", color="green")
    .background_gradient("Reds", subset=cohort_number.columns[1:], high=1, axis=None)
 )

out1

留存率,本月新增,+1月,+2月,+3月,+4月
同期群分组,,,,,
2020-12,22870,7.21%,3.55%,1.29%,0.83%
2021-01,6330,10.71%,1.93%,1.18%,0.00%
2021-02,5210,5.51%,1.65%,0.00%,0.00%
2021-03,3407,6.11%,0.00%,0.00%,0.00%
2021-04,2445,0.00%,0.00%,0.00%,0.00%


## 同期群分析 - 从人均付款金额角度

### 同期群分析

要从从人均付款金额角度考虑，需要考虑同期群基期这个整体。具体计算方式是先计算各月的付款总额，然后除以基期的总人数：

In [103]:
cohort_amount = order.pivot_table(index="同期群分组", columns="标签",
                                  values="月付费总额", aggfunc="mean",
                                  fill_value=0).rename_axis(columns="人均付款金额")
cohort_amount.insert(0, "本月新增", cohort_amount.pop("本月新增"))

In [104]:
cohort_amount = order.pivot_table(index="同期群分组", columns="标签",
                  values="月付费总额", aggfunc="sum",
                  fill_value=0).rename_axis(columns="付款总额")
cohort_amount.insert(0, "本月收入", cohort_amount.pop("本月新增"))
cohort_amount = cohort_amount.merge(cohort_number[['本月新增']],left_index=True, right_index=True)
cohort_amount.insert(0, "本月新增", cohort_amount.pop("本月新增"))
cohort_amount.iloc[:, 1:] = cohort_amount.iloc[:, 1:].divide(cohort_amount.本月新增, axis=0)
cohort_amount = cohort_amount.rename(columns={'本月收入':'本月平均消费'})
out2 = (cohort_amount.style
    .format("{:.2f}")
    .bar(subset="本月新增", color="green")
    .background_gradient("Reds",subset=cohort_amount.columns[1:],axis=None)
 )

out2

,本月新增,本月平均消费,+1月,+2月,+3月,+4月
同期群分组,,,,,,
2020-12,22870.00,703.43,89.70,61.51,11.35,11.41
2021-01,6330.00,607.94,151.43,16.28,9.76,0.00
2021-02,5210.00,661.54,73.41,20.67,0.00,0.00
2021-03,3407.00,647.90,82.46,0.00,0.00,0.00
2021-04,2445.00,557.26,0.00,0.00,0.00,0.00


可以看到，12月份的同期群首月新用户人均消费为703.43元，然后逐月递减，到+4月后这些用户人均消费仅11.41元。

而随着版本的迭代发展，新增用户的首月消费并没有较大提升，且接下来的消费趋势反而不如12月份。

由此可见产品的发展受到了一定的瓶颈，需要思考增长营收的出路了。

一般来说， 通过同期群分析可以比较好指导我们后续更深入细致的数据分析，为产品优化提供参考。

## 同期群分析 - 从人均购买次数角度

### 同期群分析

In [108]:
cohort_count = order.pivot_table(index="同期群分组", columns="标签",
                                 values="月付费次数", aggfunc="sum",
                                 fill_value=0).rename_axis(columns="人均购买次数")
cohort_count.insert(0, "本月次数", cohort_count.pop("本月新增"))
cohort_count = cohort_count.merge(cohort_number[['本月新增']],left_index=True, right_index=True)
cohort_count.insert(0, "本月新增", cohort_count.pop("本月新增"))
cohort_count.iloc[:, 1:] = cohort_count.iloc[:, 1:].divide(cohort_count.本月新增, axis=0)
cohort_count = cohort_count.rename(columns={'本月次数':'本月平均次数'})
out3 = (cohort_count.style
    .format("{:.2f}", subset=cohort_count.columns[1:])
    .bar(subset="本月新增", color="green")
    .background_gradient("Reds",subset=cohort_count.columns[1:],axis=None)
 )

out3

,本月新增,本月平均次数,+1月,+2月,+3月,+4月
同期群分组,,,,,,
2020-12,22870,2.02,0.21,0.11,0.03,0.02
2021-01,6330,1.79,0.30,0.04,0.03,0.00
2021-02,5210,1.82,0.14,0.04,0.00,0.00
2021-03,3407,1.80,0.16,0.00,0.00,0.00
2021-04,2445,1.71,0.00,0.00,0.00,0.00


### 数据分组

下面我们看看每个月的总体消费情况：

In [109]:
order.groupby("同期群分组").agg(
    付费人数=("uid", "count"),
    人均付款金额=("月付费总额", "mean"),
    月付费总额=("月付费总额", "sum")
)

,付费人数,人均付款金额,月付费总额
同期群分组,,,
2020-12,25815,777.305830,20066150
2021-01,7205,690.026371,4971640
2021-02,5583,705.137023,3936780
2021-03,3615,688.337483,2488340
2021-04,2445,557.263804,1362510


可以看到总体付费人数和付费金额都在逐月下降。

### 数据导出


将 out1、out2、out3 带格式导出至本地 html 

In [110]:
with open("out.html", "w") as f:
    f.write(out1.render())
    f.write(out2.render())
    f.write(out3.render())

以上就是本次 同期群分析 的全部内容～

![](http://liuzaoqi.oss-cn-beijing.aliyuncs.com/2021/09/16/16317972442543.jpg?域名/sample.jpg?x-oss-process=style/stylename)

![](./picture/3.png)